In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
%matplotlib inline
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:80% !important; }</style>")) 

# Load Content Embeddings

In [2]:
cwd = os.getcwd()
content_embeddings = pd.read_pickle(os.path.join(cwd, "..", "data", "autoencoder_embeddings.pkl"))
content_embeddings = pd.DataFrame(content_embeddings)
print(content_embeddings.shape)
content_embeddings.head()

(26744, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.013344,0.239560,1.050268,0.000000,0.023284,0.396033,0.000000,0.692792,0.000000,0.000000,...,0.002741,0.000000,0.265989,0.372963,0.000000,0.0,0.0,0.711748,0.000000,0.379516
1,0.000000,0.172851,1.251562,0.471290,0.065289,0.398064,0.007400,1.077674,0.143710,0.000000,...,0.000000,0.040959,0.075530,0.000000,0.000000,0.0,0.0,0.555058,0.397117,0.333755
2,0.001010,0.925408,0.063987,0.000000,0.133182,0.030352,0.104917,0.622351,0.045847,0.000000,...,0.000000,0.242329,0.000000,0.422629,0.000000,0.0,0.0,0.494329,0.182441,0.093916
3,0.000000,0.944240,0.607588,0.368766,0.166914,0.000000,0.226098,0.000000,0.481809,0.000000,...,0.000000,0.000000,0.124727,0.465437,0.014911,0.0,0.0,0.605971,0.806610,0.387506
4,0.000000,0.734243,0.627565,0.150901,0.304607,0.236698,0.051425,0.408785,0.367336,0.586794,...,0.000000,0.000000,0.083080,0.182237,0.000000,0.0,0.0,0.080750,0.826122,0.542480


# Load Collaborative Embeddings

In [3]:
cwd = os.getcwd()
collaborative_embeddings = pd.read_pickle(os.path.join(cwd, "..", "data", "movie_embeddings_1.pkl"))
print(collaborative_embeddings.shape)
collaborative_embeddings.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/paperspace/my-movie-recommender/notebooks/../data/movie_embeddings_1.pkl'

# Format Movie Lookup Data

In [4]:
# Load index mapping 
with open('../data/movie_to_idx.pkl', 'rb') as handle:
    movie2idx = pickle.load(handle)

In [5]:
movies = pd.read_csv(os.path.join(cwd, "..", "data/ml-20m", "movies.csv"))
print("{} unique movies in movies.csv".format(len(movies.movieId.unique())))

ratings = pd.read_csv(os.path.join(cwd, "..", "data/ml-20m", "ratings.csv"))
print("{} unique movies in ratings.csv".format(len(ratings.movieId.unique())))

movies = pd.merge(movies, ratings, on="movieId", how="inner")
movies.movieId = movies.movieId.apply(lambda x: movie2idx[x])

#get popularity
popularity = pd.DataFrame(movies[['userId', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['count']))
popularity.reset_index(inplace=True)
popularity.columns = ['title', 'movieId', 'ratings_count']
popularity.sort_values('ratings_count', ascending=False, inplace=True)
movies = pd.merge(popularity[['movieId', 'ratings_count']], movies, on='movieId')
movies.reset_index(inplace=True)

#get average ratings
average_ratings = pd.DataFrame(movies[['rating', 'title', 'movieId']].groupby(['title', 'movieId']).agg(['mean']))
average_ratings.reset_index(inplace=True)
average_ratings.columns = ['title', 'movieId', 'avg_rating']
movies = pd.merge(average_ratings[['movieId', 'avg_rating']], movies, on='movieId')
movies.reset_index(inplace=True)


movies = movies[['movieId', 'title', 'genres', 'ratings_count', 'avg_rating']]
movies.drop_duplicates(inplace=True)
print("{} unique movies in embeddings".format(len(movies.movieId.unique())))
movies.set_index('movieId', inplace=True, drop=True)
movies.sort_index(ascending=True, inplace=True)
print(movies.shape)
movies.head(5)

27278 unique movies in movies.csv
26744 unique movies in ratings.csv
26744 unique movies in embeddings
(26744, 4)


,title,genres,ratings_count,avg_rating
movieId,,,,
0,Jumanji (1995),Adventure|Children|Fantasy,22243,3.211977
1,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,8520,3.952230
2,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,44980,3.898055
3,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,43249,4.053493
4,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,47006,4.334372


In [7]:
movies.to_csv('../data/movie_demographics.csv')

In [23]:
movies.query('title == "Zodiac (2007)"')

,title,genres,ratings_count,avg_rating
movieId,,,,
3995,Zodiac (2007),Crime|Drama|Thriller,3907,3.675454


# Concatinate latent Tags and Embeddings

In [ ]:
from sklearn import preprocessing

#normalize ensembeled dimensions
x = np.concatenate((content_embeddings, collaborative_embeddings), axis=1)
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)
latent_df = pd.DataFrame(x_scaled)

In [ ]:
import os; import sys
cwd = os.getcwd()
path = os.path.join(cwd, '..', 'movie_recommender')
if not path in sys.path:
    sys.path.append(path)
del cwd, path

from similarity import SimilarityPredictions

In [ ]:
sim_model_ens = SimilarityPredictions(embeddings=latent_df)

In [ ]:
sim_model_cont = SimilarityPredictions(embeddings=content_embeddings)

In [ ]:
sim_model_coll = SimilarityPredictions(embeddings=collaborative_embeddings)

In [ ]:
#movie_id = 3006 #primer
#movie_id = 1195 #grease
movie_id = 131 #LOTR
#movie_id = 2087 #inception
#movie_id = 3995 #zodiac
#movie_id = 23877 #forgotton (1 rating)
#movie_id = 15816 #rated 19 times

# Ensemble recommendations

In [ ]:
output = sim_model_ens.predict_similar_items(seed_item=movie_id, n=20)

In [ ]:
similar_movies = pd.DataFrame(output)
similar_movies.set_index('item_id', inplace=True)
sim_df = pd.merge(movies, similar_movies, left_index=True, right_index=True)
sim_df.sort_values('similarity_score', ascending=False, inplace=True)
sim_df

# Content Recommendations

In [ ]:
output = sim_model_cont.predict_similar_items(seed_item=movie_id, n=20)

In [ ]:
similar_movies = pd.DataFrame(output)
similar_movies.set_index('item_id', inplace=True)
sim_df = pd.merge(movies, similar_movies, left_index=True, right_index=True)
sim_df.sort_values('similarity_score', ascending=False, inplace=True)
sim_df

# Collaborative Recommendations

In [ ]:
output = sim_model_coll.predict_similar_items(seed_item=movie_id, n=20)

In [ ]:
similar_movies = pd.DataFrame(output)
similar_movies.set_index('item_id', inplace=True)
sim_df = pd.merge(movies, similar_movies, left_index=True, right_index=True)
sim_df.sort_values('similarity_score', ascending=False, inplace=True)
sim_df